# Step 1. calibrate

In [2]:
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

# termination criteria
# https://stackoverflow.com/questions/49038464/opencv-calibrate-fisheye-lens-error-ill-conditioned-matrix
# removed cv.fisheye.CALIB_CHECK_COND 
calibration_flags = cv.fisheye.CALIB_RECOMPUTE_EXTRINSIC + + cv.fisheye.CALIB_FIX_SKEW
subpix_criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.1) # 0.001

CHECKERBOARD = (9,6)
CHECKERBOARD_SQUARE_SIDE = 22 # mm

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
# https://stackoverflow.com/questions/37310210/camera-calibration-with-opencv-how-to-adjust-chessboard-square-size
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) * CHECKERBOARD_SQUARE_SIDE

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

for fname in tqdm(Path("../data/calib_images/calib/").iterdir()):
    img = cv.imread(str(fname))

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, cv.CALIB_CB_ADAPTIVE_THRESH+cv.CALIB_CB_FAST_CHECK+cv.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        cv.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (9,6), corners, ret)
        # save image w/ chessboard to make sure everything went well
        cv.imwrite(f"../data/calib_images/processed_calib/{fname.name}", img)
    else:
        print(f'{fname}, no corners found')

15it [00:17,  2.83s/it]

../data/calib_images/calib/39.jpg, no corners found


60it [01:19,  1.32s/it]


In [3]:
# calibrate
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for _ in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for _ in range(N_OK)]

rms, _, _, _, _ = \
    cv.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv.TERM_CRITERIA_EPS+cv.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

Found 59 valid images for calibration
K=np.array([[807.001663913809, 0.0, 1769.57653582959], [0.0, 806.2636130746708, 1241.9479742423312], [0.0, 0.0, 1.0]])
D=np.array([[-0.010379570060940762], [-0.0031983087526026065], [0.0003114367284975261], [-0.0002890556350962741]])


In [ ]:
# make sure the calibration went well: are straight lines straight
img = cv.imread("../data/calib_images/vp_calib.png")
# img = cv.imread("../data/calib_images/pnp_3264x2464.png")

# 1_real.jpg is 1280x720, resize to 2560x1440 and pad to 3264x2464
# resize the image to 2560x1440
# img = center_crop(img, (2560, 1440))
# img = cv.resize(img, (2560, 1440))

# # # calculate padding dimensions
# pad_y = (3264 - img.shape[1]) // 2  # vertical padding
# pad_x = (2464 - img.shape[0]) // 2  # horizontal padding

# # padding the image to make it 3264x2464
# img = np.pad(
#     img, 
#     ((pad_y, pad_y), (pad_x, pad_x), (0, 0)), 
#     mode='constant', 
#     constant_values=0
# )
new_K = cv.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, (3264, 2464), np.eye(3), balance=1.0)
map1, map2 = cv.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, (3264,2464), cv.CV_16SC2)
undistorted_img = cv.remap(img, map1, map2, interpolation=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT)
cv.imwrite("undistorted_vp.jpg", undistorted_img)
plt.imshow(undistorted_img)

In [17]:
np.save("K_3264x2464.npy", K)
np.save("D_3264x2464.npy", D)

In [14]:
K

array([[8.07001664e+02, 0.00000000e+00, 1.76957654e+03],
       [0.00000000e+00, 8.06263613e+02, 1.24194797e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# Step 2. Find camera pose
alright unsure how to make this work, vp calib seems fine? and we can add roll as augmentations and lezgo

In [ ]:
# pnp find camera pose

def draw(img, corners, imgpts):
    # lines
    corner = tuple(corners[0].ravel())
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)

    # # cube
    # imgpts = np.int32(imgpts).reshape(-1,2)
    # # draw ground floor in green
    # img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # # draw pillars in blue color
    # for i,j in zip(range(4),range(4,8)):
    #     img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # # draw top layer in red color
    # img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    
    return img

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3) * CHECKERBOARD_SQUARE_SIDE # bc squares are 22 mm wide
# axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
#                    [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ]) * 22

img = cv.imread("../data/calib_images/pnp_3264x2464.png")

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

ret, corners = cv.findChessboardCorners(gray, (9,6),None)
if ret == True:
    corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),subpix_criteria)
    cv.drawChessboardCorners(img, (9,6), corners2, ret)
    # Find the rotation and translation vectors.
    ret, rvecs, tvecs = cv.fisheye.solvePnP(objp,corners2, K, D)
    # rvecs, tvecs = cv.solvePnPRefineVVS(objp, corners2, K, D, rvecs, tvecs)
    if ret:
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, K, D)
        img = draw(img,corners2.astype(np.uint),imgpts.astype(np.uint))
        cv.imwrite("pnp.jpg", img)
        plt.imshow(img)
    else:
        print("pnp didn't work")
else:
    print("didn't find")

In [96]:
np.rad2deg(rvecs)

array([[-57.16322434],
       [ 53.1469279 ],
       [ 75.67984408]])

In [94]:
tvecs/10

array([[ 6.40850352],
       [ 9.36371699],
       [58.603831  ]])

In [ ]:
# cube doesn't look well aligned?
# and tvecs z say 5.8 cm when cam is actually about 12cm above ground
# this seems off
# is is the calibration that's wrong?
# a mistake on my end?

In [1]:
import numpy as np

K = np.load("K_3264x2464.npy")
D = np.load("D_3264x2464.npy")

# Step 3. Distort sim images

To make sim images look like real ones we want to:
- crop them like the camera/software does
- distort them the same way the lens does
- match the simulated camera pose to the real camera pose on the car
- mask the car hood and lens

In [18]:
import torch
from typing import Optional
from kornia.geometry.calibration import distort_points
import numpy as np

def distort_image(image: torch.Tensor, K: np.ndarray, D: np.ndarray) -> torch.Tensor:
    # Get image shape
    h, w = image.size(1), image.size(2)


    # Generate grid of points in the image
    x = torch.linspace(0, w-1, w)
    y = torch.linspace(0, h-1, h)
    grid_y, grid_x = torch.meshgrid(y, x)
    points = torch.stack([grid_x.t(), grid_y.t()], dim=2)  # t() is for transpose

    # Distort the points
    # cv2
    # distorted_points = cv.fisheye.distortPoints(points.numpy(), K, D)
    # distorted_points = torch.from_numpy(distorted_points)

    # kornia
    K = torch.from_numpy(K)
    D = torch.from_numpy(D)
    distorted_points = distort_points(points, K, D)

    # Apply the mapping using grid_sample. Note that grid_sample expects the grid to be in the range [-1, 1] and the points to be in the format (x, y), so we have to do some conversions
    grid = (distorted_points - torch.tensor([[w/2, h/2]])) / torch.tensor([[w/2, h/2]])  # normalization and centering
    grid = grid.permute(2, 0, 1).unsqueeze(0)  # add batch dimension
    grid = grid.permute(0, 3, 2, 1)  # reorder dimensions

    # Interpolate
    distorted_image = torch.nn.functional.grid_sample(image.unsqueeze(0), grid.float(), align_corners=False, padding_mode="zeros")

    return distorted_image.squeeze(0)


In [10]:
import torch
import cv2 as cv
from torchvision.io import read_image, write_png
import torchvision.transforms.functional as F

# now can we distort sim images
# img = read_image("../data/sim_v_real/1_sim.jpg").float()
img = cv.imread("../data/sim_v_real/1_tiny.jpg") # 816x616

rescaled_K = np.copy(K)
rescaled_K = rescaled_K/4
rescaled_K[2][2] = 1

print(img.shape)
dist = distort_image(img, rescaled_K, D[:, 0], crop_output=False, crop_type="middle")
print(dist.shape)
cv.imwrite("sim_distorted.jpg", dist)

(616, 816, 3)
(616, 816, 3)


True

In [6]:
import matplotlib.pyplot as plt

In [103]:
dist

array([[-0.57353866,  0.23593632,  0.00098039, -0.00281957, -0.03686973]])

In [ ]:
# rendering 1, 2, 3, 4 in blender with eevee and cycles
# coords (x, y, z)
# 1: 2.98999 m, 0.472049 m
# 2: 0.88 m, 1.51205 m 
# 3: 0.12 m, 1.51205 m
# 4: 6.57 m, 2.69205 m


In [162]:
# center crop 3264x2464 images to 2560x1440 (which is how 720p images are cropped)
def center_crop(img, dim):
	"""Returns center cropped image
	Args:
	img: image to be center cropped
	dim: dimensions (width, height) to be cropped
	"""
	width, height = img.shape[1], img.shape[0]

	# process crop width and height for max available dimension
	crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
	crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
	mid_x, mid_y = int(width/2), int(height/2)
	cw2, ch2 = int(crop_width/2), int(crop_height/2) 
	crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
	return crop_img